* 软件开发领域最经典的口头禅就是“don't repeat yourself”。解决代码重复的问题可以使用元编程，即创建关于操作源代码得函数和类，主要用到的技术有装饰器、类装饰器、元类，还有其他的诸如：签名对象、使用exec（）执行代码以及对内部函数和类进行反射技术等

# 9.1在函数上添加包装器
* 问题：想在函数上添加一个包装器，增加额外的操作处理（如日志、计时等）
* 方案：如果想要使用额外的代码包装一个函数可以使用一个装饰器函数

In [1]:
import time 
from functools import wraps
def timethis(func):
    '''一个报告执行时间的装饰器'''
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__,end-start)
        return result
    return wrapper
#下面使用装饰器
@timethis
def countdown(n):
    while n > 0:
        n -= 1

In [2]:
countdown(100000)

countdown 0.009973287582397461


* 一个装饰器就是一个函数，他接受一个函数作为参数，并返回一个新的函数

In [3]:
@timethis
def countdown(n):
    pass
#上面的代码和下面得一样
def countdown(n):
    pass
countdown = timethis(countdown)

* 内置的装饰器比如：@staticmethod，@classmethod，@property原理也是一样的

In [4]:
class A:
    @classmethod
    def method(cls):
        pass
class B:
    def method(cls):
        pass
    method = classmethod(method)
#上述得两段代码的功能也是一样的

# 9.2 创建装饰器时保留函数元信息
* 问题：写了一个装饰器作用在某个函数上，但是这个函数的重要的元信息比如名字、文档字符串、注解和参数签名都丢失了。
* 方案：任何时候你定义装饰器的时候，都应该使用 functools 库中的 @wraps 装饰器来注解底层包装函数

In [62]:
import time
from functools import wraps
def timethis(func):
    '''
    Decorator that reports the execution time.
    '''
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

In [63]:
@timethis
def countdown(n):
    '''
    Counts down
    '''
    while n > 0:
        n -= 1
        

In [64]:
countdown(10000)

countdown 0.0009970664978027344


In [65]:
countdown.__name__

'countdown'

In [66]:
countdown.__doc__

'\n    Counts down\n    '

In [67]:
countdown.__annotations__

{}

* @wraps 有一个重要特征是它能让你通过属性 \_\_wrapped\_\_ 直接访问被包装函数

In [68]:
countdown.__wrapped__(10000)

* \_\_wrapped\_\_ 属性还能让被装饰函数正确暴露底层的参数签名信息

In [69]:
from inspect import signature
print(signature(countdown))

(n)


* 在编写装饰器的时候复制元信息是一个非常重要的部分。如果你忘记了使用 @wraps
，那么你会发现被装饰函数丢失了所有有用的信息。：

In [57]:
import time
from functools import wraps
def timethis(func):
    '''
    Decorator that reports the execution time.
    '''
    #@wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

@timethis
def countdown(n):
    '''
    Counts down
    '''
    while n > 0:
        n -= 1

In [58]:
countdown.__name__

'wrapper'

In [59]:
countdown.__doc__

# 9.3解除一个装饰器
* 问题：一个装饰器已经作用在一个函数上，想撤销他直接访问未被包装的函数
* 方案：假设装饰器是通过@wraps实现的，可以通过访问\_\_wrapped\_\_属性访问原始函数

In [81]:
def D(func):
    @wraps(func)
    def Print(*args):
        print('hello')
        return func(*args)
    return Print
@D
def add(x,y):
    print('add')
    return x+y
add(3,4)

hello
add


7

In [82]:
orig_add = add.__wrapped__
orig_add(3,4)

add


7

* 并不是所有的装饰器都使用了@wraps，特别的，内置的@staticmethod和@classmethod就把原始函数的属性存储在\_\_func\_\_

# 9.4定义一个带参数的装饰器
* 问题：装饰器可以接受参数
* 方案：如下，写了一个装饰器，给函数添加日志功能，同时允许用户指定日志的级别和其他选项

In [84]:
from functools import wraps
import logging
def logged(level, name=None, message=None):
    """
    Add logging to a function. level is the logging
    level, name is the logger name, and message is the
    log message. If name and message aren't specified,
    they default to the function's module and name.
    """
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level,logmsg)
            return func(*args, **kwargs)
        return wrapper
    return decorate
#使用
@logged(logging.DEBUG)
def add(x,y):
    return x + y
@logged(logging.CRITICAL,'example')
def spam():
    print('spam!')
            

* 上面的代码最外层的函数 logged()接受参数并将它们作用在内部的装饰器函数上面。内层的函数 decorate() 接受一个函
数作为参数，然后在函数上面放置一个包装器。这里的关键点是包装器是可以使用传递
给 logged() 的参数的。

In [86]:
#下面的两段代码的处理过程是一样的
# @decorator(x,y,z)
# def func(a,b):
#     pass

# def func(a,b):
#     pass
# func = decorator(x,y,z)(func)

# 9.5可自定义属性的装饰器
* 问题：写一个装饰器来包装一个函数，并且允许用户提供参数在运行时控制装饰器的行为
* 方案：引入一个访问函数，使用nonlocal来修改内部变量，然后该访问函数作为一个属性赋值给包装函数

In [88]:
from functools import wraps, partial
import logging
def attach_wrapper(obj, func=None):
    if func is None:
        return partial(attach_wrapper,obj)
    setattr(obj,func.__name__,func)
    return func
def logged(level,name=None,message=None):
    """
    Add logging to a function. level is the logging
    level, name is the logger name, and message is the
    log message. If name and message aren't specified,
    they default to the function's module and name.
    """
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level,logmsg)
            return func(*args, **kwargs)
        
        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel
            
        @attach_wrapper(wrapper)
        def set_message(newmsg):
            nonlocal message
            message = newmsg
            
        return wrapper
    return decorate

In [90]:
#使用的例子
@logged(logging.DEBUG)
def add(x,y):
    return x+y

@logged(logging.CRITICAL,'example')
def spam():
    print('spam!')

In [91]:
logging.basicConfig(level=logging.DEBUG)
add(2,3)

DEBUG:__main__:add


5

In [92]:
#change the log message
add.set_message('Add called')
add(2,3)

DEBUG:__main__:add


5

In [93]:
#change the log level
add.set_level(logging.WARNING)
add(2,3)

5

* 该小节的关键点在于访问函数（set_message()和set_level()），它们被作为属性赋值给包装器，每个访问函数允许使用nonlocal来修改函数内部的变量
* 还有一个惊奇的地方就是访问函数会在多层装饰器之间传播

#  9.6带可选参数的装饰器
* 问题：想写一个装饰器，既可以传入参数，又可以不传参数
* 方案：如下

In [97]:
from functools import wraps,partial
import logging
def logged(func=None,*, level=logging.DEBUG, name=None, message=None):
    if func is None:
        return partial(logged,level=level, name=name, message=message)
    logname = name if name else func.__module__
    log = logging.getLogger(logname)
    logmsg = message if message else func.__name__
    @wraps(func)
    def wrapper(*args, **kwargs):
        log.log(level,logmsg)
        return func(*args, **kwargs)
    return wrapper
    
#使用1
@logged
def add(x,y):
    return x+y

#使用2
@logged(level=logging.CRITICAL,name='example')
def spam():
    print("spam!")

* 可以看到上述代码@logged既可以带参数，又可以不带参数

In [100]:
#使用1  和下面的方式相同
def add(x,y):
    return x+y
add = logged(add)
#被装饰函数会被当做第一个参数直接传递给 logged 装饰器。
#因此，logged() 中的第一个参数就是被包装函数本身。所有其他参数都必须有默认值。

In [105]:
#使用2   和下面的方式相同
def spam():
    print('spam!')
spam = logged(level=logging.CRITICAL,name='example')(spam)
# 初始调用 logged() 函数时，被包装函数并没有传递进来。因此在装饰器内，它必
# 须是可选的。这个反过来会迫使其他参数必须使用关键字来指定。并且，但这些参数被
# 传递进来后，装饰器要返回一个接受一个函数参数并包装它的函数

# 9.7利用装饰器强制函数上的类型检查
* 问题：作为某种变成规约，相对函数参数进行强制类型检查
* 方案：如下

In [108]:
#使用装饰器实现@typeassert
from inspect import signature
from functools import wraps
def typeassert(*ty_args, **ty_kwargs):
    def decorate(func):
        # if in optimized mode,disable type checking
        if not __debug__:
            return func
        # map function argument names to supplied types
        sig = signature(func)
        bound_types = sig.bind_partial(*ty_args, **ty_kwargs).arguments
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            bound_values = sig.bind(*args,**kwargs)
            for name,value in bound_values.arguments.items():
                if name in bound_types:
                    if not isinstance(value, bound_types[name]):
                        raise TypeError('argument {} must be {}'.format(name,bound_types[name]))
            return func(*args, **kwargs)
        return wrapper
    return decorate

* 这个装饰器非常灵活，既可以指定所有参数类型，也可以只指定部分。并
且可以通过位置或关键字来指定参数类型。

In [109]:
@typeassert(int,z=int)
def spam(x,y,z=42):
    print(x,y,z)
spam(1,2,3)

1 2 3


In [111]:
spam(1,'hello',9)

1 hello 9


In [110]:
spam(1,'hello','word')

TypeError: argument z must be <class 'int'>

* 首先装饰器只会在函数定义的时候被调用一次
* 另外使用inspect.signature()函数来提取一个可调用对象的参数签名信息。

In [112]:
from inspect import signature
def spam(x,y,z=42):
    pass
sig = signature(spam)
print(sig)

(x, y, z=42)


In [113]:
sig.parameters

mappingproxy({'x': <Parameter "x">,
              'y': <Parameter "y">,
              'z': <Parameter "z=42">})

In [116]:
sig.parameters['z'].name

'z'

In [117]:
sig.parameters['z'].default

42

In [118]:
sig.parameters['z'].kind

<_ParameterKind.POSITIONAL_OR_KEYWORD: 1>

* 使用bind_partial()方法来执行从指定类型到名称的部分绑定

In [119]:
bound_types = sig.bind_partial(int,z=int)
bound_types

<BoundArguments (x=<class 'int'>, z=<class 'int'>)>

In [120]:
bound_types.arguments

OrderedDict([('x', int), ('z', int)])

* bind()和bind_partial()功能相似，但是它不允许缺失任何参数。上面的代码中缺失了对y的绑定，但不会报错。

In [121]:
bound_values = sig.bind(1,2,3)
bound_values.arguments

OrderedDict([('x', 1), ('y', 2), ('z', 3)])